### Demonstration: Constructing the `AlleleProfile` Class

This notebook demonstrates the construction of the `AlleleProfile` class, a Python implementation of the HL7 FHIR MolecularDefinition profile.

#### Overview
The `AlleleProfile` class, built with Pydantic, extends the `MolecularDefinition` class to meet the specific requirements of the HL7 FHIR Allele Profile. It incorporates tailored attributes and behaviors to support this use case.


### Step 1: Prerequisites and Setup

To construct the `AlleleProfile` class in this demonstration, we set up the environment by importing the specific libraries and modules required for the examples in this notebook. These include:

1. **FHIR Resources**:
   - `Coding`, `CodeableConcept`, `Quantity`, `Reference`, and `Meta` from the `fhir.resources` library. These classes provide the framework for working with structured healthcare data in FHIR format.

2. **Custom Project Modules**:
   - `AlleleProfile` from `profiles.alleleprofile`: The main class we will build and demonstrate.
   - Components from `moldefresource.moleculardefinition`:
     - `MolecularDefinitionLocation` and related classes for specifying molecular location.
     - `MolecularDefinitionRepresentation` and `MolecularDefinitionRepresentationLiteral` for defining molecular representations.

By importing these libraries, we ensure access to all the tools needed for this demonstration.


In [14]:
# Import required libraries
from fhir.resources.coding import Coding
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.quantity import Quantity
from fhir.resources.reference import Reference
from fhir.resources.meta import Meta
from profiles.alleleprofile import AlleleProfile
from moldefresource.moleculardefinition import (
    MolecularDefinitionLocation,
    MolecularDefinitionLocationSequenceLocation,
    MolecularDefinitionLocationSequenceLocationCoordinateInterval,
    MolecularDefinitionLocationSequenceLocationCoordinateIntervalCoordinateSystem,
    MolecularDefinitionRepresentation,
    MolecularDefinitionRepresentationLiteral
)


### Step 2: Defining the MolecularDefinition Location Component

In this step, we define the location component for the `MolecularDefinition` resource.

Key steps include:
1. **Coordinate System**: Create a `Coding` instance and wrap it in a `CodeableConcept`.
2. **Interval and Sequence Location**: Build the interval with start and end quantities.
3. **Sequence Context**: Link to the reference molecular definition using a `Reference`.
4. **Final Structure**: Combine these into a `MolecularDefinitionLocation`.

#### Example Workflow:
The JSON location is implemented in Python as follows in next cell:
```json
{
  "location": [
    {
      "sequenceLocation": {
        "sequenceContext": {
          "reference": "MolecularDefinition/example-sequence-nc000002-url",
          "type": "MolecularDefinition",
          "display": "NC_000002.12"
        },
        "coordinateInterval": {
          "coordinateSystem": {
            "system": {
              "coding": [
                {
                  "system": "http://loinc.org",
                  "code": "LA30100-4",
                  "display": "0-based interbase"
                }
              ],
              "text": "0-based interbase"
            }
          },
          "startQuantity": {
            "value": 27453448
          },
          "endQuantity": {
            "value": 27453449
          }
        }
      }
    }
  ]
}

In [15]:
# Step 2: MolecularDefinition Location Component
coding_val = Coding(
    system="http://loinc.org",
    code="LA30100-4",
    display="0-based interbase",
)

codeconcept_val = CodeableConcept(
    coding=[coding_val],
    text="0-based interbase",
)

MolDefLocSeqLocCoordIntCoord = MolecularDefinitionLocationSequenceLocationCoordinateIntervalCoordinateSystem(
    system=codeconcept_val
)

MolDefLocSeqLocCoordInt = MolecularDefinitionLocationSequenceLocationCoordinateInterval(
    coordinateSystem=MolDefLocSeqLocCoordIntCoord,
    startQuantity=Quantity(value=27453448),
    endQuantity=Quantity(value=27453449),
)

MolDefLocSeqLoc = MolecularDefinitionLocationSequenceLocation(
    sequenceContext=Reference(
        reference="MolecularDefinition/example-sequence-nc000002-url",
        type="MolecularDefinition",
        display="NC_000002.12",
    ),
    coordinateInterval=MolDefLocSeqLocCoordInt,
)

MolDefLoc = MolecularDefinitionLocation(
    sequenceLocation=MolDefLocSeqLoc
)

# Inspect serialized output
MolDefLoc.model_dump()


{'sequenceLocation': {'sequenceContext': {'reference': 'MolecularDefinition/example-sequence-nc000002-url',
   'type': 'MolecularDefinition',
   'display': 'NC_000002.12'},
  'coordinateInterval': {'coordinateSystem': {'system': {'coding': [{'system': 'http://loinc.org',
       'code': 'LA30100-4',
       'display': '0-based interbase'}],
     'text': '0-based interbase'}},
   'startQuantity': {'value': Decimal('27453448')},
   'endQuantity': {'value': Decimal('27453449')}}}}

### Step 3: Defining the MolecularDefinition Representation Component

In this step, we define the representation component for the `MolecularDefinition` resource.

#### Key Steps:
1. **Literal Representation**: Define the literal value using `MolecularDefinitionRepresentationLiteral`.
2. **Focus Coding**: Create a `Coding` instance to specify the focus, and wrap it in a `CodeableConcept`.
3. **Combine Components**: Integrate these into the `MolecularDefinitionRepresentation`.

#### JSON Representation:
```json
{
  "representation": [
    {
      "focus": {
        "coding": [
          {
            "system": "http://hl7.org/fhir/moleculardefinition-focus",
            "code": "allele-state",
            "display": "Allele State"
          }
        ]
      },
      "literal": {
        "value": "T"
      }
    }
  ]
}


In [17]:
MolDefRepLit = MolecularDefinitionRepresentationLiteral(value="T")

focus_coding_val = Coding(
        system="http://hl7.org/fhir/moleculardefinition-focus",
        code= "allele-state",
        display= "Allele State"
    )

focus_codeconcept_val = CodeableConcept(
    coding=[focus_coding_val],
)

MolDefRep = MolecularDefinitionRepresentation(
    literal=MolDefRepLit,
    focus=focus_codeconcept_val
    )
# Inspect serialized output
MolDefRep.model_dump()


{'focus': {'coding': [{'system': 'http://hl7.org/fhir/moleculardefinition-focus',
    'code': 'allele-state',
    'display': 'Allele State'}]},
 'literal': {'value': 'T'}}

### Step 4: Integrating Components into the Complete `AlleleProfile`

In this step, we combine all the previously defined components to create a complete instance of the `AlleleProfile` class. This integration includes metadata, molecule type, and the location and representation components.

#### Key Features of the `AlleleProfile`:
- **Meta**: Specifies the profile URL to ensure compliance with HL7 FHIR standards.
- **Molecule Type**: Defines the type of sequence, such as DNA, RNA, or protein.
- **Location and Representation**: Incorporates the `MolecularDefinitionLocation` and `MolecularDefinitionRepresentation` components created earlier.

#### JSON Representation:
```json
{
  "resourceType": "MolecularDefinition",
  "id": "demo-example-hgvs-substitition",
  "meta": {
    "profile": ["http://hl7.org/fhir/StructureDefinition/allelesliced"]
  },
  "moleculeType": {
    "coding": [
      {
        "system": "http://hl7.org/fhir/sequence-type",
        "code": "dna",
        "display": "DNA Sequence"
      }
    ]
  },
  "location": [
    {...}  // Details from MolecularDefinitionLocation
  ],
  "representation": [
    {...}  // Details from MolecularDefinitionRepresentation
  ]
}


In [18]:
meta_val = Meta(
  profile= ["http://hl7.org/fhir/StructureDefinition/allelesliced"]
)
moltype_coding_val = Coding(
        system="http://hl7.org/fhir/sequence-type",
        code= "dna",
        display= "DNA Sequence"
    )
moltype_codeconcept_val = CodeableConcept(
    coding=[moltype_coding_val],
)
Allele_Example = AlleleProfile(
  id = 'demo-example-hgvs-substitition',
  meta = meta_val,
  moleculeType=moltype_codeconcept_val,
  location=[MolDefLoc],
  representation=[MolDefRep]
)
# Inspect serialized output
Allele_Example.model_dump()


{'resourceType': 'MolecularDefinition',
 'id': 'demo-example-hgvs-substitition',
 'meta': {'profile': ['http://hl7.org/fhir/StructureDefinition/allelesliced']},
 'moleculeType': {'coding': [{'system': 'http://hl7.org/fhir/sequence-type',
    'code': 'dna',
    'display': 'DNA Sequence'}]},
 'location': [{'sequenceLocation': {'sequenceContext': {'reference': 'MolecularDefinition/example-sequence-nc000002-url',
     'type': 'MolecularDefinition',
     'display': 'NC_000002.12'},
    'coordinateInterval': {'coordinateSystem': {'system': {'coding': [{'system': 'http://loinc.org',
         'code': 'LA30100-4',
         'display': '0-based interbase'}],
       'text': '0-based interbase'}},
     'startQuantity': {'value': Decimal('27453448')},
     'endQuantity': {'value': Decimal('27453449')}}}}],
 'representation': [{'focus': {'coding': [{'system': 'http://hl7.org/fhir/moleculardefinition-focus',
      'code': 'allele-state',
      'display': 'Allele State'}]},
   'literal': {'value': 'T'}

### Final Example: Using JSON to Generate the Class

Instead of constructing the `AlleleProfile` step by step, we can take the complete JSON structure and directly generate the class instance using Python's `**` unpacking syntax. This method simplifies the process and ensures the class instance accurately reflects the provided JSON data.

In [20]:
# Example 1 - Substitition origin: "NC_000002.12:g.27453449C>T"
example_substitition ={
    "resourceType" : "MolecularDefinition",
    "id" : "demo-example-hgvs-substitition",
    "meta" : {
      "profile" : ["http://hl7.org/fhir/StructureDefinition/allelesliced"]
    },
    "moleculeType" : {
      "coding" : [{
        "system" : "http://hl7.org/fhir/sequence-type",
        "code" : "dna",
        "display" : "DNA Sequence"
      }]
    },
    "location" : [
        {
      "sequenceLocation" : {
        "sequenceContext" : {
          "reference" : "MolecularDefinition/example-sequence-nc000002-url",
          "type" : "MolecularDefinition",
          # Example needs to contain the reference sequence
          "display" : "NC_000002.12"
        },
        "coordinateInterval" : {
          "coordinateSystem" : {
            "system" : {
              "coding" : [{
                "system" : "http://loinc.org",
                "code" : "LA30100-4",
                # Example needs to contain the systems coordinate
                "display" : "0-based interbase" 
              }],
              "text" : "0-based interbase"
            }
          },
          # Example needs to contain the startQuantity
          "startQuantity" : {
            "value" : 27453448
          },
          # Example needs to contain the endQuantity
          "endQuantity" : {
            "value" : 27453449
          }
        }
      }
    }
    ],
    "representation" : [{
      "focus" : {
        "coding" : [{
          "system" : "http://hl7.org/fhir/moleculardefinition-focus",
          "code" : "allele-state",
          "display" : "Allele State"
        }]
      },
      "literal" : {
        "value" : "T"
      }
    }]
  }

AlleleProfile(**example_substitition).model_dump()

{'resourceType': 'MolecularDefinition',
 'id': 'demo-example-hgvs-substitition',
 'meta': {'profile': ['http://hl7.org/fhir/StructureDefinition/allelesliced']},
 'moleculeType': {'coding': [{'system': 'http://hl7.org/fhir/sequence-type',
    'code': 'dna',
    'display': 'DNA Sequence'}]},
 'location': [{'sequenceLocation': {'sequenceContext': {'reference': 'MolecularDefinition/example-sequence-nc000002-url',
     'type': 'MolecularDefinition',
     'display': 'NC_000002.12'},
    'coordinateInterval': {'coordinateSystem': {'system': {'coding': [{'system': 'http://loinc.org',
         'code': 'LA30100-4',
         'display': '0-based interbase'}],
       'text': '0-based interbase'}},
     'startQuantity': {'value': Decimal('27453448')},
     'endQuantity': {'value': Decimal('27453449')}}}}],
 'representation': [{'focus': {'coding': [{'system': 'http://hl7.org/fhir/moleculardefinition-focus',
      'code': 'allele-state',
      'display': 'Allele State'}]},
   'literal': {'value': 'T'}